# AUTOMATIC1111 Stable Diffusion WebUI 1.5 + ChilloutMix Checkpoint

More info: https://github.com/wibus-wee/stable_diffusion_chilloutmix_ipynb

> Created by [@wibus-wee](https://github.com/wibus-wee)
>
> Reference: [camenduru/stable-diffusion-webui-colab](https://github.com/camenduru/stable-diffusion-webui-colab)

In [ ]:
#@title Select Checkpoint - 选择模型 { display-mode: "form" }

checkpoint = 'chilloutmix.safetensors' #@param ["chilloutmix.safetensors", "sunshinemix.safetensors", "grapefruitHentaiModel.safetensors"]

downloadLink = {
    'chilloutmix.safetensors': 'https://civitai.com/api/download/models/11745',
    'sunshinemix.safetensors': 'https://civitai.com/api/download/models/11752',
    'grapefruitHentaiModel.safetensors': 'https://civitai.com/api/download/models/9000'
}[checkpoint]

print("已选择模型: " + checkpoint + " <===> " + downloadLink)

%store checkpoint
%store downloadLink

In [ ]:
#@title 2. Check GPU & Check Environment - 检查 GPU & 检查环境

#@markdown This step will check if your GPU supports xformers, and check if you are on Paperspace (only M4000 GPU is checked, so other GPUs may have a logic error here, you may need to check it yourself).
#@markdown 此步骤将检查你的 GPU 是否支持 xformers，同时会检查是否为 Paperspace 平台（仅针对 M4000 GPU 做了判断，因此其他的 GPU 可能在此处将会出现逻辑判断错误的情况，你可能需要自行勾选）。

import os, subprocess
paperspace_m4000 = False
#@markdown 是否为 Paperspace 平台
isPaperspace = False #@param {type:"boolean"}
try:
    subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE)
    if 'M4000' in subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8'):
        print("WARNING: 你正在使用的是 Quadro M4000 GPU，它将无法使用 xformers。")
        paperspace_m4000 = True
        isPaperspace = True
    else:
        print("你正在使用的是合适的 GPU - " + subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8') + "。")
        print("平台: Paperspace" if isPaperspace else "使用平台: Colab")
except:
    print("似乎没有 GPU 可用。请检查你的运行时类型。")
    exit()

rootDir = isPaperspace and '/tmp' or '/content'
stableDiffusionWebUIInstalled = os.path.exists(rootDir + '/stable-diffusion-webui')
%store rootDir
%store paperspace_m4000 
%store isPaperspace
%store stableDiffusionWebUIInstalled

In [ ]:
#@title 3. Install Dependencies & Start - 第一次使用 - 安装依赖并启动

%store -r paperspace_m4000 
%store -r isPaperspace
%store -r rootDir 
%store -r checkpoint 
%store -r downloadLink
%store -r stableDiffusionWebUIInstalled

import os, subprocess
from glob import glob
from pathlib import Path
import sys

!apt-get -y install -qq aria2
ariaInstalled = False

try:
    subprocess.run(['aria2c', '--version'], stdout=subprocess.PIPE)
    ariaInstalled = True
except:
    pass

if paperspace_m4000:
  # Uninstall PyTorch and some other libraries so the WebUI can install the versions it needs
  !pip uninstall -y torch torchvision torchaudio protobuf
  !pip install protobuf
  !pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
else:
  !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.16/xformers-0.0.16+814314d.d20230118-cp38-cp38-linux_x86_64.whl
  !pip install -q --pre triton
  

!git clone -b v2.0 https://github.com/camenduru/stable-diffusion-webui {rootDir}/stable-diffusion-webui
!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O {rootDir}/stable-diffusion-webui/scripts/run_n_times.py
!git clone https://github.com/deforum-art/deforum-for-automatic1111-webui {rootDir}/stable-diffusion-webui/extensions/deforum-for-automatic1111-webui
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser {rootDir}/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
!git clone https://github.com/camenduru/stable-diffusion-webui-huggingface {rootDir}/stable-diffusion-webui/extensions/stable-diffusion-webui-huggingface
!git clone -b v2.0 https://github.com/camenduru/sd-civitai-browser {rootDir}/stable-diffusion-webui/extensions/sd-civitai-browser
!git clone https://github.com/kohya-ss/sd-webui-additional-networks {rootDir}/stable-diffusion-webui/extensions/sd-webui-additional-networks

if isPaperspace:
  %cd /stable-diffusion-webui
else:
  %cd {rootDir}/stable-diffusion-webui
  

if ariaInstalled:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {downloadLink} -d {rootDir}/stable-diffusion-webui/models/Stable-diffusion -o {checkpoint}
else:
    !wget -c {downloadLink} -P {rootDir}/stable-diffusion-webui/models/Stable-diffusion -O {rootDir}/stable-diffusion-webui/models/Stable-diffusion/{checkpoint}

stableDiffusionWebUIInstalled = True
%store stableDiffusionWebUIInstalled

%cd {rootDir}/stable-diffusion-webui
!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/self.logvar\\[t\\]/self.logvar\\[t.item()\\]/g\"'' {rootDir}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/models/diffusion/ddpm.py""")''' {rootDir}/stable-diffusion-webui/launch.py
!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' {rootDir}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' {rootDir}/stable-diffusion-webui/launch.py
!python launch.py --share --enable-insecure-extension-access {'' if paperspace_m4000 else '--xformers'}

In [ ]:

#@title 4. Restart Stable Diffusion WebUI  - 重新启动 Stable Diffusion WebUI

%store -r paperspace_m4000 
%store -r rootDir 
%store -r stableDiffusionWebUIInstalled

if not stableDiffusionWebUIInstalled:
    print("请先运行安装 Stable Diffusion WebUI 的单元格。")
    exit()

%cd {rootDir}/stable-diffusion-webui
!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/self.logvar\\[t\\]/self.logvar\\[t.item()\\]/g\"'' {rootDir}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/models/diffusion/ddpm.py""")''' {rootDir}/stable-diffusion-webui/launch.py
!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' {rootDir}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' {rootDir}/stable-diffusion-webui/launch.py
!python launch.py --share --enable-insecure-extension-access {'' if paperspace_m4000 else '--xformers'}

In [ ]:
#@title 5. Export generated images - 导出生成的图片
%store -r rootDir 

from pathlib import Path
import os, subprocess

export_storage_dir = Path(rootDir, 'export')
export_storage_dir.mkdir(exist_ok=True)

!if [ $(dpkg-query -W -f='${Status}' p7zip-full 2>/dev/null | grep -c "ok installed") = 0 ]; then sudo apt update && sudo apt install -y p7zip-full; fi # install 7z if it isn't already installed
from datetime import datetime
datetime_str = datetime.now().strftime('%m-%d-%Y_%H-%M-%S')
%cd "{export_storage_dir}"
!mkdir -p "{datetime_str}/log"
!cd "{rootDir}/stable-diffusion-webui/log" && mv * "{export_storage_dir / datetime_str / 'log'}"
!cd "{rootDir}/stable-diffusion-webui/outputs" && mv * "{export_storage_dir / datetime_str}"
s = subprocess.run(f'find "{Path(export_storage_dir, datetime_str)}" -type d -name .ipynb_checkpoints -exec rm -rv {{}} +', shell=True)
!7z a -t7z -m0=lzma2 -mx=9 -mfb=64 -md=32m -ms=on "{datetime_str}.7z" "{export_storage_dir / datetime_str}"

In [ ]:
#@title 6. Install and launch when you need to train a model - 需要训练模型时安装与启动

#@markdown But but according to what I know now, its sd_dreambooth_extension doesn't work very well. Now I recommend you use [bmaltais/kohya_ss](https://github.com/bmaltais/kohya_ss)
#@markdown 但是根据我目前了解的信息，其 sd_dreambooth_extension 无法很好地工作。现在我更推荐你使用 [bmaltais/kohya_ss](https://github.com/bmaltais/kohya_ss)
%store -r paperspace_m4000 
%store -r isPaperspace
%store -r rootDir 
%store -r checkpoint 
%store -r downloadLink
%store -r stableDiffusionWebUIInstalled

trainExtensionInstalled = False


import os, subprocess
from glob import glob
from pathlib import Path
import sys

if not stableDiffusionWebUIInstalled:
  !apt-get -y install -qq aria2
  ariaInstalled = False

  try:
      subprocess.run(['aria2c', '--version'], stdout=subprocess.PIPE)
      ariaInstalled = True
  except:
      pass

  if paperspace_m4000:
    # Uninstall PyTorch and some other libraries so the WebUI can install the versions it needs
    !pip uninstall -y torch torchvision torchaudio protobuf
    !pip install protobuf
    !pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
  else:
    !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.16/xformers-0.0.16+814314d.d20230118-cp38-cp38-linux_x86_64.whl
    !pip install -q --pre triton
    

  !git clone -b v2.0 https://github.com/camenduru/stable-diffusion-webui {rootDir}/stable-diffusion-webui
  !wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O {rootDir}/stable-diffusion-webui/scripts/run_n_times.py
  !git clone https://github.com/deforum-art/deforum-for-automatic1111-webui {rootDir}/stable-diffusion-webui/extensions/deforum-for-automatic1111-webui
  !git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser {rootDir}/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
  !git clone https://github.com/camenduru/stable-diffusion-webui-huggingface {rootDir}/stable-diffusion-webui/extensions/stable-diffusion-webui-huggingface
  !git clone -b v2.0 https://github.com/camenduru/sd-civitai-browser {rootDir}/stable-diffusion-webui/extensions/sd-civitai-browser
  !git clone https://github.com/kohya-ss/sd-webui-additional-networks {rootDir}/stable-diffusion-webui/extensions/sd-webui-additional-networks

!git clone https://github.com/d8ahazard/sd_dreambooth_extension {rootDir}/stable-diffusion-webui/extensions/sd_dreambooth_extension
trainExtensionInstalled = True

if isPaperspace:
  %cd /stable-diffusion-webui
else:
  %cd {rootDir}/stable-diffusion-webui
  
if ariaInstalled:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {downloadLink} -d {rootDir}/stable-diffusion-webui/models/Stable-diffusion -o {checkpoint}
else:
    !wget -c {downloadLink} -P {rootDir}/stable-diffusion-webui/models/Stable-diffusion -O {rootDir}/stable-diffusion-webui/models/Stable-diffusion/{checkpoint}

%cd {rootDir}/stable-diffusion-webui
!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/self.logvar\\[t\\]/self.logvar\\[t.item()\\]/g\"'' {rootDir}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/models/diffusion/ddpm.py""")''' {rootDir}/stable-diffusion-webui/launch.py
!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' {rootDir}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' {rootDir}/stable-diffusion-webui/launch.py
!export REQS_FILE="./extensions/sd_dreambooth_extension/requirements.txt"
!python launch.py --share --enable-insecure-extension-access {'' if paperspace_m4000 else '--xformers'}

In [ ]:
#@title 7. Install and launch when you need to train a model - 需要训练模型时安装与启动
%store -r paperspace_m4000 
%store -r rootDir 
%store -r trainExtensionInstalled
%store -r stableDiffusionWebUIInstalled

if not stableDiffusionWebUIInstalled:
    print("请先运行安装 Stable Diffusion WebUI 的单元格。")
    exit()

if not trainExtensionInstalled:
    print("请先运行安装训练扩展的单元格。")
    exit()

%cd {rootDir}/stable-diffusion-webui
!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/self.logvar\\[t\\]/self.logvar\\[t.item()\\]/g\"'' {rootDir}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/models/diffusion/ddpm.py""")''' {rootDir}/stable-diffusion-webui/launch.py
!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' {rootDir}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' {rootDir}/stable-diffusion-webui/launch.py
!export REQS_FILE="./extensions/sd_dreambooth_extension/requirements.txt"
!python launch.py --share --enable-insecure-extension-access {'' if paperspace_m4000 else '--xformers'}